In [1]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

from utils.linkedin import scrape_linkedin_profile

import os
#import requests

from key_storage import key_dict
list(key_dict.keys())

['OPENAI_API_KEY', 'PINECONE_API_KEY', 'PINECONE_API_ENV', 'PROXYCURL_API_KEY']

In [2]:
info,data = scrape_linkedin_profile("https://www.linkedin.com/in/venetis-pallikaras-9b6bb1135/",key_dict["PROXYCURL_API_KEY"])
data

{'public_identifier': 'venetis-pallikaras',
 'profile_pic_url': 'https://media.licdn.com/dms/image/C4D03AQFnf7IY3H_GXA/profile-displayphoto-shrink_400_400/0/1644887767960?e=1689811200&v=beta&t=-C-4BINLMZPdcdvZ3Atf1oDxJRE_srR8U2R6kfekod8',
 'background_cover_image_url': None,
 'first_name': 'Venetis',
 'last_name': 'Pallikaras',
 'full_name': 'Venetis Pallikaras',
 'follower_count': None,
 'occupation': 'Graduate Teaching Assistant at Georgia Institute of Technology',
 'headline': 'Data Science | Machine Learning',
 'summary': None,
 'country': 'US',
 'country_full_name': 'United States of America',
 'city': 'Washington DC-Baltimore Area',
 'state': None,
 'experiences': [{'starts_at': {'day': 1, 'month': 8, 'year': 2022},
   'ends_at': None,
   'company': 'Georgia Institute of Technology',
   'company_linkedin_profile_url': 'https://www.linkedin.com/school/georgia-institute-of-technology/',
   'title': 'Graduate Teaching Assistant',
   'description': 'GTA for the course CS 7641-Machine

In [3]:
info

{'public_identifier': 'venetis-pallikaras',
 'first_name': 'Venetis',
 'last_name': 'Pallikaras',
 'full_name': 'Venetis Pallikaras',
 'follower_count': 'no info',
 'occupation': 'Graduate Teaching Assistant at Georgia Institute of Technology',
 'headline': 'Data Science | Machine Learning',
 'summary': 'no info',
 'country_full_name': 'United States of America',
 'city': 'Washington DC-Baltimore Area',
 'experiences': [{'starts_at': {'day': 1, 'month': 8, 'year': 2022},
   'ends_at': None,
   'company': 'Georgia Institute of Technology',
   'company_linkedin_profile_url': 'https://www.linkedin.com/school/georgia-institute-of-technology/',
   'title': 'Graduate Teaching Assistant',
   'description': 'GTA for the course CS 7641-Machine Learning',
   'location': 'Atlanta, Georgia, United States',
   'logo_url': 'https://media.licdn.com/dms/image/C4E0BAQFuhLQoh5_7yA/company-logo_400_400/0/1640110531895?e=1692230400&v=beta&t=uRZjSl7Dxj4cNW8zdyU7tZukLyDhvBW7w9qxYzWZJhw'},
  {'starts_at': {'

In [4]:
info,data = scrape_linkedin_profile("https://www.linkedin.com/in/venetis-pallikaras-broken/",key_dict["PROXYCURL_API_KEY"])
info,data 

couldn't retrieve information-1
<PreparedRequest [GET]>
<Response [404]>


(None, None)

In [5]:
info,data = scrape_linkedin_profile("https://www.linkedin.com/in/brokenlinktest/",key_dict["PROXYCURL_API_KEY"])
info,data 

couldn't retrieve information-1
<PreparedRequest [GET]>
<Response [404]>


(None, None)

In [6]:
info,data = scrape_linkedin_profile("https://httpbin.org/status/403",key_dict["PROXYCURL_API_KEY"])
info,data 

couldn't retrieve information-1
<PreparedRequest [GET]>
<Response [400]>


(None, None)

In [4]:
summary_template = """
         given the information about a person from linkedin {information},I want you to create:
         1. a short summary
         2. two interesting facts about them
         \n
     """

llm = ChatOpenAI(
                 temperature=0,
                 model_name = "gpt-3.5-turbo",
                 openai_api_key = key_dict["OPENAI_API_KEY"],
                 )

summary_prompt_template = PromptTemplate(
        input_variables=["information"],
        template=summary_template,

    )

chain = LLMChain(llm=llm, prompt=summary_prompt_template)

In [5]:
chain.run(information=info)

'1. Venetis Pallikaras is a Graduate Teaching Assistant at Georgia Institute of Technology, with a focus on Data Science and Machine Learning. They have also completed several courses and certifications in related fields.\n2. Venetis has interned at GE and NCSR "DEMOKRITOS" in Greece, where they worked on electrical characterization of memristive devices and analyzed the behavior of memristors under various conditions. They have also completed a Data Analyst Nanodegree and several courses on PySpark and TensorFlow.'